# Business Analytics - Assignment 2 - Question 2
#### **Student Name:** Koorosh Shakoori

## Glue Company

In this task we are solving a Fixed Charge IP problem using PULP.

In [26]:
from pulp import *

## Defining the Decision Variables

The decision variables in this problem are two fold:
- First set of decision variables are the number of workers for each line. This variable is of Integer type and can vary between 0 and 7.
- The second set of decision variables are the decision to either set up a specific line for production or not. This Variables is of Binary type and the equivalent element of the first set is dependant on this variable.
    - 1 if the line is set up
    - 0 if the line not set up

In [27]:
Lines = [
    'Line_1',
    'Line_2'
]
Workers_Fee = {
    'Line_1': 500,
    'Line_2': 900
}

Setup_Fee = {
    'Line_1': 1000,
    'Line_2': 2000
}

Glue_1 = {
    'Line_1': 20,
    'Line_2': 50
}

Glue_2 = {
    'Line_1': 30,
    'Line_2': 35
}

Glue_3 = {
    'Line_1': 40,
    'Line_2': 45
}

Workers_variables = LpVariable.dicts('Workers', Lines, 0, cat=LpInteger)
Setup_variables = LpVariable.dicts('Setup', Lines, 0, cat=LpBinary)

## Defining the Objective Function
Given the fee to employ a worker in each line is set by the problem and setting up each line has an initial cost, we will have a function for the cost of production which we need to minimize according to the needs of the problem.
Therefore, the objective function is defined as below:

In [28]:
#First we will intialize a LP problem object from pulp with the objective to minimize to fit our objective described below.
prob = LpProblem('Course_Plan', LpMinimize)

#Now we will add our objective function which is the cost of setting up the lines and hiring workers for each line
prob += lpSum([Workers_variables[line]*Workers_Fee[line] + Setup_variables[line]*Setup_Fee[line] for line in Lines])

## Defining the Constraints
Now we set Constraints as required in the assignments, we need to address:
- The first three constraints is for satisfying the demands for each type of glue on a weekly basis:
    - Type 1 Glue -> At least 120
    - Type 2 Glue -> At least 150
    - Type 3 Glue -> At least 200
- The maximum number of workers in each line can be 7.
- The next two constraints are to assure that the problem solver understands that if one line is used it needs to be setup and paid for. We use the BigM variable for this Purpose.

In [29]:
#For the first three constraints we use the glue dictionaries that we defined which show the production of glue per worker in each line:
prob += lpSum([Workers_variables[line]*Glue_1[line] for line in Lines]) >= 120, 'Glue_1_Demand'
prob += lpSum([Workers_variables[line]*Glue_2[line] for line in Lines]) >= 150, 'Glue_2_Demand'
prob += lpSum([Workers_variables[line]*Glue_3[line] for line in Lines]) >= 200, 'Glue_3_Demand'

BigM = 10
for line in Lines:
    #For these constraints we only need to address the number of workers:
    prob += Workers_variables[line] <= 7, f'{line}_Maximum_Capacity'
    
    #The next two constraints will use BigM concept to stop our solver to assign production in lanes that are not set up and paid for:
    prob += Workers_variables[line] - BigM*Setup_variables[line] <= 0, f'{line}_Setup_Control'


In [30]:
#Here we get an overall report on all sides of the problem
prob

Course_Plan:
MINIMIZE
1000*Setup_Line_1 + 2000*Setup_Line_2 + 500*Workers_Line_1 + 900*Workers_Line_2 + 0
SUBJECT TO
Glue_1_Demand: 20 Workers_Line_1 + 50 Workers_Line_2 >= 120

Glue_2_Demand: 30 Workers_Line_1 + 35 Workers_Line_2 >= 150

Glue_3_Demand: 40 Workers_Line_1 + 45 Workers_Line_2 >= 200

Line_1_Maximum_Capacity: Workers_Line_1 <= 7

Line_1_Setup_Control: - 10 Setup_Line_1 + Workers_Line_1 <= 0

Line_2_Maximum_Capacity: Workers_Line_2 <= 7

Line_2_Setup_Control: - 10 Setup_Line_2 + Workers_Line_2 <= 0

VARIABLES
0 <= Setup_Line_1 <= 1 Integer
0 <= Setup_Line_2 <= 1 Integer
0 <= Workers_Line_1 Integer
0 <= Workers_Line_2 Integer

In [31]:
prob.solve(PULP_CBC_CMD(msg=0))

1

In [32]:
#The optimal hiring and setup profile is shown below.
#The total weekly cost of the process is also mentioned.
for var in prob.variables():
    print(f'{var.name} = {var.varValue}')
print(f'The total weekly production cost = {prob.objective.value()}')

Setup_Line_1 = 1.0
Setup_Line_2 = 0.0
Workers_Line_1 = 6.0
Workers_Line_2 = 0.0
The total weekly production cost = 4000.0


## Conclusions
The numbers above explain the optimal solution which shows **only line number 1 needs to be setup** with **6 workers hired** to meet the weekly demand.

The cost of production in this case will be **4000** which is the least amount of money to meet the demand.
